In [ ]:
import os
import h5py
import numpy as np
import logging
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd

# Physics-related Packages
from astropy.cosmology import Planck15
# Physics-related Packages
from astropy.cosmology import Planck15

In [ ]:
snapnum = 84
FilePath = f"/users_path/merger_trace/data/tng_cluster/tng_cluster_groupcat/groupcat_0{snapnum}/"
SavePath = f"/users_path/merger_trace/data/tng_cluster/tng_cluster_targetcat/targethalo_cat_0{snapnum}/"

In [ ]:
# preparations for read box info from the url
import requests

baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"API KEY"}

def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [ ]:
# Issue a request to the API root
r = get(baseUrl)

# Print out all the simulation names
names = [sim['name'] for sim in r['simulations']]
# Get the index of TNG300-1
i = names.index('TNG-Cluster')
# Get the info of simulation Illustris-3
sim = get( r['simulations'][i]['url'] )
sim.keys()

# get the snaps info this simulation
snaps = get(sim['snapshots'])

In [ ]:
# Sim Box parameters
Snap_Index = snapnum # the snapshots index in the total 100 snapshots taking at different z
BoxSize = sim['boxsize'] # unit: ckpc/h
Redshift = snaps[Snap_Index]['redshift'] # current redshift of our current snap

# Short description of cosmological parameters Planck2015
h_atz = Planck15.H(Redshift).value/100 # unit 100km/[s*Mpc]

# h should be h_0, but it does not make a difference, so I don't change the codes here
LowerMass_lim = 10**3 * h_atz # units 10^ 10 𝑀⊙/h
HigherMass_lim = 10**10 * h_atz # units 10^ 10 𝑀⊙/h

print(h_atz)

In [ ]:
# Configure file paths and log file
log_file_path = "file_check_log.txt"# Basic Package s

chunk_numbers = range(0, 600)  # Assume chunk_number ranges from 0 to 599

# Open the log file for writing
with open(log_file_path, "w") as log_file:
    log_file.write("File Check Log\n")
    log_file.write("================\n")

    # Iterate through all chunk_numbers
    for chunk_number in chunk_numbers:
        file_name = FilePath + f'fof_subhalo_tab_0{snapnum}.{chunk_number}.hdf5'

        # Check if the file exists
        if os.path.isfile(file_name):
            try:
                # Try opening the HDF5 file to check if it is readable
                with h5py.File(file_name, "r") as hdf_file:
                    log_file.write(f"{file_name}: File exists and is a valid HDF5 file.\n")
            except Exception as e:
                log_file.write(f"{file_name}: File exists but is not a valid HDF5 file. Error: {e}\n")
        else:
            log_file.write(f"{file_name}: File does not exist.\n")

print(f"File check completed. Results are saved in {log_file_path}")

In [ ]:
### Delete the Previous File in the Directory

directory = SavePath

if os.path.exists(directory):
    # loop over all files/directories in the target directory
    for file_or_dir in os.listdir(directory):
        file_or_dir_path = os.path.join(directory, file_or_dir)
        try:
            if os.path.isfile(file_or_dir_path) or os.path.islink(file_or_dir_path):
                os.remove(file_or_dir_path)  
                print(f"{file_or_dir_path} has already been deleted")
            elif os.path.isdir(file_or_dir_path):
                shutil.rmtree(file_or_dir_path)  
                print(f"{file_or_dir_path} has already been deleted")
        except Exception as e:
            print(f"cannot remove {file_or_dir_path}: {e}")
else:
    print(f"The directory {directory} does not exist")

In [ ]:
# create the null files for sky zones
# Dict names used here are similar to those in Illustris

Target_Halo_Catalog = f'{SavePath}TargetHalo_MergerCat_0{snapnum}.hdf5'

with h5py.File(Target_Halo_Catalog, 'w') as hdf:
    # create dict for fof halos
    Group = hdf.create_group('Group')

    Group.create_dataset(
        'GroupCM',
        shape = (0,3),
        maxshape = (None,3),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,3)
    
    Group.create_dataset(
        'GroupFirstSub',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    Group.create_dataset(
        'GroupLenType',
        shape = (0,6),
        maxshape = (None,6),
        dtype = 'float64'
        ) # unit: None, Dims: (N,6)
    
    Group.create_dataset(
        'GroupMassType',
        shape = (0,6),
        maxshape = (None,6),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,6)
    
    Group.create_dataset(
        'GroupNsubs',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    Group.create_dataset(
        'GroupPos',
        shape = (0,3),
        maxshape = (None,3),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,3)
    
    Group.create_dataset(
        'GroupSFR',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: M_{sun}/yr, Dims: (N,)
    
    Group.create_dataset(
        'Group_M_Crit200',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,)
    
    Group.create_dataset(
        'Group_M_Crit500',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,)
    
    Group.create_dataset(
        'Group_M_Mean200',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,)
    
    Group.create_dataset(
        'Group_R_Crit200',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,)
    
    Group.create_dataset(
        'Group_R_Crit500',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,)
    
    Group.create_dataset(
        'Group_R_Mean200',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,)
    
    Group.create_dataset(
        'FOF_Halo_IDs',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    # create dict for subhalos
    Subhalo = hdf.create_group('Subhalo')
    
    Subhalo.create_dataset(
        'SubhaloFlag',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloCM',
        shape = (0,3),
        maxshape = (None,3),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,3)
    
    Subhalo.create_dataset(
        'SubhaloGrNr',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloHalfmassRad',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloLenType',
        shape = (0,6),
        maxshape = (None,6),
        dtype = 'float64'
        ) # unit: None, Dims: (N,6)
    
    Subhalo.create_dataset(
        'SubhaloMassInRadType',
        shape = (0,6),
        maxshape = (None,6),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,6)
    

    Subhalo.create_dataset(
        'SubhaloMassInRad',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloMass',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,)

    Subhalo.create_dataset(
        'SubhaloMassInHalfRad',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloMassType',
        shape = (0,6),
        maxshape = (None,6),
        dtype = 'float64'
        ) # unit: 10^10 M_{sun}/h, Dims: (N,6)
    
    Subhalo.create_dataset(
        'SubhaloParent',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloPos',
        shape = (0,3),
        maxshape = (None,3),
        dtype = 'float64'
        ) # unit: ckpc/h, Dims: (N,3)
    
    Subhalo.create_dataset(
        'SubhaloSFR',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: M_{sun}/yr, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloSFRinHalfRad',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: M_{sun}/yr, Dims: (N,)

    Subhalo.create_dataset(
        'SubhaloSFRinRad',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: M_{sun}/yr, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloVelDisp',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: km/s, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloVel',
        shape = (0,3),
        maxshape = (None,3),
        dtype = 'float64'
        ) # unit: km/s, Dims: (N,3)
    
        
    Subhalo.create_dataset(
        'SubhaloSpin',
        shape = (0,3),
        maxshape = (None,3),
        dtype = 'float64'
        ) # unit: ckpc/h* km/s, Dims: (N,3)
    
    Subhalo.create_dataset(
        'Subhalo_IDs',
        shape = (0,),
        maxshape = (None,),
        dtype = 'float64'
        ) # unit: None, Dims: (N,)
    
    Subhalo.create_dataset(
        'SubhaloStellarPhotometrics',
        shape = (0,8),
        maxshape = (None,8),
        dtype = 'float64'
        ) # mag

In [ ]:
def get_centersub(Halo_IDs, Sub_GrNr, Sub_Mass):
    
    """
    Finds the indices of the top three most massive subhalos for a given halo.

    Parameters:
    - Halo_ID: Array of halo IDs (single value or array with specific halo ID to match).
    - Sub_GrNr: Array indicating the group number (halo) each subhalo belongs to.
    - Sub_Mass: Array of subhalo masses.

    Returns:
    - Numpy array (CenterSub_Index, SecondSub_Index, ThirdSub_Index):
      - CenterSub_Index: Index of the most massive subhalo in the halo.
      - SecondSub_Index: Index of the second most massive subhalo.
      - ThirdSub_Index: Index of the third most massive subhalo.
    """
    CenterSub_Indices = np.zeros(Halo_IDs.shape)
    CenterSub_Masses = np.zeros(Halo_IDs.shape)

    for i in range(len(Halo_IDs)):
        Halo_ID = Halo_IDs[i]
        find_Sub = np.where(Sub_GrNr == Halo_ID)[0]
        if len(find_Sub)!=0:
          find_Sub_Mass = Sub_Mass[find_Sub]
          find_Sub_Mass_Sorted = np.argsort(find_Sub_Mass)

          CenterSub_Index = np.where(Sub_GrNr == Halo_ID)[0][find_Sub_Mass_Sorted[-1]]
          CenterSub_Indices[i] = np.array(CenterSub_Index)
          CenterSub_Masses[i] = Sub_Mass[CenterSub_Index]
        else:
          CenterSub_Indices[i] = -1
          CenterSub_Masses[i] = 0
    
    return CenterSub_Indices, CenterSub_Masses

In [ ]:
# Sort Target Halo Catalog
def sort_halos(
        snap_number, chunk_number, 
        FilePath, SavePath,
        Last_Halo_IDs, Last_Subhalo_IDs):
    '''
    Last_Halo_IDs is the number of fof halos in the sorted chunks,
    Last_Subhalo_IDs is the number of subhalos in the sorted chunks
    '''
    groupcat_name = f'{FilePath}fof_subhalo_tab_0{snap_number}.{chunk_number}.hdf5'

    with h5py.File(groupcat_name, 'r') as cat_hdf:
        if not 'Group/GroupCM' in cat_hdf:
            print(f"Warning: {groupcat_name} is empty. Skipping data reading.")
            num_Halo_thiscat = 0
            num_subhalo_thiscat = 0
        else:
            # get info for groups
            GroupCM = cat_hdf['Group/GroupCM'][:]
            GroupFirstSub = cat_hdf['Group/GroupFirstSub'][:]
            GroupLenType = cat_hdf['Group/GroupLenType'][:]
            GroupMassType = cat_hdf['Group/GroupMassType'][:]
            GroupNsubs = cat_hdf['Group/GroupNsubs'][:]
            GroupPos = cat_hdf['Group/GroupPos'][:]
            GroupSFR = cat_hdf['Group/GroupSFR'][:]
            Group_M_Crit200 = cat_hdf['Group/Group_M_Crit200'][:]
            Group_M_Crit500 = cat_hdf['Group/Group_M_Crit500'][:]
            Group_M_Mean200 = cat_hdf['Group/Group_M_Mean200'][:]
            Group_R_Crit200 = cat_hdf['Group/Group_R_Crit200'][:]
            Group_R_Crit500 = cat_hdf['Group/Group_R_Crit500'][:]
            Group_R_Mean200 = cat_hdf['Group/Group_R_Mean200'][:]
            FOF_Halo_IDs = np.arange(Last_Halo_IDs, Last_Halo_IDs + GroupNsubs.shape[0])
            num_Halo_thiscat = GroupNsubs.shape[0]
            print('num_halo_thiscat',num_Halo_thiscat)
            
            # get info for subhalos
            SubhaloFlag = cat_hdf['Subhalo/SubhaloFlag'][:]
            SubhaloCM = cat_hdf['Subhalo/SubhaloCM'][:]
            SubhaloGrNr = cat_hdf['Subhalo/SubhaloGrNr'][:]
            SubhaloHalfmassRad = cat_hdf['Subhalo/SubhaloHalfmassRad'][:]
            SubhaloLenType = cat_hdf['Subhalo/SubhaloLenType'][:]
            SubhaloMassInRadType = cat_hdf['Subhalo/SubhaloMassInRadType'][:]
            SubhaloMassInRad = cat_hdf['Subhalo/SubhaloMassInRad'][:]
            SubhaloMassInHalfRad = cat_hdf['Subhalo/SubhaloMassInHalfRad'][:]
            SubhaloMass = cat_hdf['Subhalo/SubhaloMass'][:]
            SubhaloMassType = cat_hdf['Subhalo/SubhaloMassType'][:]
            SubhaloParent = cat_hdf['Subhalo/SubhaloParent'][:]
            SubhaloPos = cat_hdf['Subhalo/SubhaloPos'][:]
            SubhaloSFR = cat_hdf['Subhalo/SubhaloSFR'][:]
            SubhaloSFRinHalfRad = cat_hdf['Subhalo/SubhaloSFRinHalfRad'][:]
            SubhaloSFRinRad = cat_hdf['Subhalo/SubhaloSFRinRad'][:]
            SubhaloSpin = cat_hdf['Subhalo/SubhaloSpin'][:]
            SubhaloVelDisp = cat_hdf['Subhalo/SubhaloVelDisp'][:]
            SubhaloVel = cat_hdf['Subhalo/SubhaloVel'][:]
            SubhaloStellarPhotometrics = cat_hdf['Subhalo/SubhaloStellarPhotometrics'][:]
            Subhalo_IDs = np.arange(Last_Subhalo_IDs, Last_Subhalo_IDs + SubhaloFlag.shape[0])
            num_subhalo_thiscat = SubhaloFlag.shape[0]
            print('num_subhalo_thiscat',num_subhalo_thiscat)
            Centersub_Masses = np.array(get_centersub(FOF_Halo_IDs, SubhaloGrNr, SubhaloMass)[1])
    
            TargetHalo_inCurrentChunk = np.array(np.where((Centersub_Masses >= LowerMass_lim) & 
                           (Centersub_Masses <= HigherMass_lim))[0])
    
            print(TargetHalo_inCurrentChunk)
    
            TargetCat = f'{SavePath}TargetHalo_MergerCat_0{snapnum}.hdf5'
            with h5py.File(TargetCat, 'a') as Target_hdf:
    
                if TargetHalo_inCurrentChunk.shape[0] > 0:
                    
                    # resize the original datasets to add data conveniently
                    # get the new size
                    num_exist_group = Target_hdf['Group/GroupFirstSub'].shape[0]
                    num_add_group = TargetHalo_inCurrentChunk.shape[0]
                    new_num_group = num_exist_group + num_add_group
                    
                    # resize
                    Target_hdf['Group/GroupCM'].resize((new_num_group,3))
                    Target_hdf['Group/GroupFirstSub'].resize((new_num_group,))
                    Target_hdf['Group/GroupLenType'].resize((new_num_group,6))
                    Target_hdf['Group/GroupMassType'].resize((new_num_group,6))
                    Target_hdf['Group/GroupNsubs'].resize((new_num_group,))
                    Target_hdf['Group/GroupPos'].resize((new_num_group,3))
                    Target_hdf['Group/GroupSFR'].resize((new_num_group,))
                    Target_hdf['Group/Group_M_Crit200'].resize((new_num_group,))
                    Target_hdf['Group/Group_M_Crit500'].resize((new_num_group,))
                    Target_hdf['Group/Group_M_Mean200'].resize((new_num_group,))
                    Target_hdf['Group/Group_R_Crit200'].resize((new_num_group,))
                    Target_hdf['Group/Group_R_Crit500'].resize((new_num_group,))
                    Target_hdf['Group/Group_R_Mean200'].resize((new_num_group,))
                    Target_hdf['Group/FOF_Halo_IDs'].resize((new_num_group,))
                    # use slices command to add data
                    Target_hdf['Group/GroupCM'][num_exist_group:new_num_group,:] = GroupCM[TargetHalo_inCurrentChunk].reshape(-1,3)
                    
                    Target_hdf['Group/GroupFirstSub'][num_exist_group:new_num_group] = GroupFirstSub[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/GroupLenType'][num_exist_group:new_num_group,:] = GroupLenType[TargetHalo_inCurrentChunk].reshape(-1,6)
                    
                    Target_hdf['Group/GroupMassType'][num_exist_group:new_num_group,:] = GroupMassType[TargetHalo_inCurrentChunk].reshape(-1,6)
                    
                    Target_hdf['Group/GroupNsubs'][num_exist_group:new_num_group] = GroupNsubs[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/GroupPos'][num_exist_group:new_num_group,:] = GroupPos[TargetHalo_inCurrentChunk].reshape(-1,3)
                    
                    Target_hdf['Group/GroupSFR'][num_exist_group:new_num_group] = GroupSFR[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/Group_M_Crit200'][num_exist_group:new_num_group] = Group_M_Crit200[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/Group_M_Crit500'][num_exist_group:new_num_group] = Group_M_Crit500[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/Group_M_Mean200'][num_exist_group:new_num_group] = Group_M_Mean200[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/Group_R_Crit200'][num_exist_group:new_num_group] = Group_R_Crit200[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/Group_R_Crit500'][num_exist_group:new_num_group] = Group_R_Crit500[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/Group_R_Mean200'][num_exist_group:new_num_group] = Group_R_Mean200[TargetHalo_inCurrentChunk].reshape(-1)
                    
                    Target_hdf['Group/FOF_Halo_IDs'][num_exist_group:new_num_group] = FOF_Halo_IDs[TargetHalo_inCurrentChunk].reshape(-1)

                    del num_exist_group, num_add_group, new_num_group

                    print('Finished FOF part')

                    # Find Subhalos belong to the target halos
                    for FOF_Halo_ID in FOF_Halo_IDs[TargetHalo_inCurrentChunk].reshape(-1):
                        find_Sub = np.array(np.where(SubhaloGrNr == FOF_Halo_ID)[0])
                        
                        print(f"Processing FOF_Halo_ID: {FOF_Halo_ID}, find_Sub.shape[0]: {find_Sub.shape[0]}")
                        if find_Sub.shape[0]>0:
                            num_exist_subhalo = Target_hdf['Subhalo/SubhaloFlag'].shape[0]
                            num_add_subhalo = find_Sub.shape[0]
                            new_num_subhalo = num_exist_subhalo + num_add_subhalo
                            # resize
                            Target_hdf['Subhalo/SubhaloFlag'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloCM'].resize((new_num_subhalo,3))
                            Target_hdf['Subhalo/SubhaloGrNr'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloHalfmassRad'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloLenType'].resize((new_num_subhalo,6))
                            Target_hdf['Subhalo/SubhaloMassInRadType'].resize((new_num_subhalo,6))
                            Target_hdf['Subhalo/SubhaloMassInRad'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloMassInHalfRad'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloMass'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloMassType'].resize((new_num_subhalo,6))
                            Target_hdf['Subhalo/SubhaloParent'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloPos'].resize((new_num_subhalo,3))
                            Target_hdf['Subhalo/SubhaloSFR'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloSFRinHalfRad'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloSFRinRad'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloSpin'].resize((new_num_subhalo,3))
                            Target_hdf['Subhalo/SubhaloVel'].resize((new_num_subhalo,3))
                            Target_hdf['Subhalo/SubhaloVelDisp'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/Subhalo_IDs'].resize((new_num_subhalo,))
                            Target_hdf['Subhalo/SubhaloStellarPhotometrics'].resize((new_num_subhalo,8))
                            
                            #use slices command to add data
                            Target_hdf['Subhalo/SubhaloFlag'][num_exist_subhalo:new_num_subhalo] = SubhaloFlag[find_Sub].reshape(-1)
                                                                                                
                            Target_hdf['Subhalo/SubhaloCM'][num_exist_subhalo:new_num_subhalo,:] = SubhaloCM[find_Sub].reshape(-1,3)
                                                                                                
                            Target_hdf['Subhalo/SubhaloGrNr'][num_exist_subhalo:new_num_subhalo] = SubhaloGrNr[find_Sub].reshape(-1)
                                                                                                
                            Target_hdf['Subhalo/SubhaloHalfmassRad'][num_exist_subhalo:new_num_subhalo] = SubhaloHalfmassRad[find_Sub].reshape(-1)
                                                                                                
                            Target_hdf['Subhalo/SubhaloLenType'][num_exist_subhalo:new_num_subhalo,:] = SubhaloLenType[find_Sub].reshape(-1,6)
                                                                                                
                            Target_hdf['Subhalo/SubhaloMassInRadType'][num_exist_subhalo:new_num_subhalo,:] = SubhaloMassInRadType[find_Sub].reshape(-1,6)

                            Target_hdf['Subhalo/SubhaloMassInRad'][num_exist_subhalo:new_num_subhalo] = SubhaloMassInRad[find_Sub].reshape(-1)

                            Target_hdf['Subhalo/SubhaloMassInHalfRad'][num_exist_subhalo:new_num_subhalo] = SubhaloMassInHalfRad[find_Sub].reshape(-1)

                            Target_hdf['Subhalo/SubhaloMass'][num_exist_subhalo:new_num_subhalo] = SubhaloMass[find_Sub].reshape(-1)
                                                                                                
                            Target_hdf['Subhalo/SubhaloMassType'][num_exist_subhalo:new_num_subhalo,:] = SubhaloMassType[find_Sub].reshape(-1,6)
                                                                                                
                            Target_hdf['Subhalo/SubhaloParent'][num_exist_subhalo:new_num_subhalo] = SubhaloParent[find_Sub].reshape(-1)
                                                                                                
                            Target_hdf['Subhalo/SubhaloPos'][num_exist_subhalo:new_num_subhalo,:] = SubhaloPos[find_Sub].reshape(-1,3)
                                                                                                
                            Target_hdf['Subhalo/SubhaloSFR'][num_exist_subhalo:new_num_subhalo] = SubhaloSFR[find_Sub].reshape(-1)

                            Target_hdf['Subhalo/SubhaloSFRinHalfRad'][num_exist_subhalo:new_num_subhalo] = SubhaloSFRinHalfRad[find_Sub].reshape(-1)

                            Target_hdf['Subhalo/SubhaloSFRinRad'][num_exist_subhalo:new_num_subhalo] = SubhaloSFRinRad[find_Sub].reshape(-1)

                            Target_hdf['Subhalo/SubhaloSpin'][num_exist_subhalo:new_num_subhalo,:] = SubhaloSpin[find_Sub].reshape(-1,3)

                            Target_hdf['Subhalo/SubhaloVelDisp'][num_exist_subhalo:new_num_subhalo] = SubhaloVelDisp[find_Sub].reshape(-1)

                            Target_hdf['Subhalo/SubhaloVel'][num_exist_subhalo:new_num_subhalo] = SubhaloVel[find_Sub].reshape(-1,3)
                            
                            Target_hdf['Subhalo/SubhaloStellarPhotometrics'][num_exist_subhalo:new_num_subhalo] = SubhaloStellarPhotometrics[find_Sub].reshape(-1,8)
                                                                                                
                            Target_hdf['Subhalo/Subhalo_IDs'][num_exist_subhalo:new_num_subhalo] = Subhalo_IDs[find_Sub].reshape(-1)
                            print('write subhalo info')
                                                                                            
                        del num_exist_subhalo, num_add_subhalo, new_num_subhalo
            
            del (GroupCM, GroupFirstSub, GroupLenType, GroupMassType,
            GroupNsubs, GroupPos, GroupSFR, Group_M_Crit200, Group_M_Crit500, 
            Group_M_Mean200, Group_R_Crit200, Group_R_Crit500, Group_R_Mean200,
            FOF_Halo_IDs, SubhaloFlag, SubhaloCM, SubhaloGrNr, SubhaloHalfmassRad,
            SubhaloLenType, SubhaloMassInRadType, SubhaloMassType, SubhaloParent,
            SubhaloPos, SubhaloSFR, Subhalo_IDs, SubhaloMassInRad, SubhaloSFRinHalfRad,
            SubhaloSFRinRad, SubhaloSpin, SubhaloVelDisp
            )
        
    gc.collect()
    print(f"Returning from sort_halos: num_Halo_thiscat={num_Halo_thiscat}, num_subhalo_thiscat={num_subhalo_thiscat}")
    return num_Halo_thiscat, num_subhalo_thiscat

In [ ]:
with h5py.File(f'{FilePath}fof_subhalo_tab_0{snapnum}.0.hdf5', 'r') as cat_hdf:
    if 'Subhalo/SubhaloMassInRad' in cat_hdf:
        SubhaloSpin = cat_hdf['Subhalo/SubhaloSpin'][:]
        print(SubhaloSpin)
    else:
        print("Warning: SubhaloMassInRad dataset is missing!")
        SubhaloMassInRad = None

In [ ]:
# Sort Target Halo Catalog
def find_total_numbers(
        snap_number, chunk_number, 
        FilePath):
    '''
    Last_Halo_IDs is the number of fof halos in the sorted chunks,
    Last_Subhalo_IDs is the number of subhalos in the sorted chunks
    '''
    groupcat_name = f'{FilePath}fof_subhalo_tab_0{snap_number}.{chunk_number}.hdf5'

    with h5py.File(groupcat_name, 'r') as cat_hdf:
        if not 'Group/GroupCM' in cat_hdf:
            print(f"Warning: {groupcat_name} is empty. Skipping data reading.")
            num_Halo_thiscat = 0
            num_subhalo_thiscat = 0
        else:
            # Read Group Number
            GroupNsubs = cat_hdf['Group/GroupNsubs'][:]
            num_Halo_thiscat = GroupNsubs.shape[0]
            # Read Subhalo Number
            SubhaloFlag = cat_hdf['Subhalo/SubhaloFlag'][:]
            num_subhalo_thiscat = SubhaloFlag.shape[0]
            
    return num_Halo_thiscat, num_subhalo_thiscat

In [ ]:
# Generate logging info when sorting particles
log_file = f'{SavePath}sort_halos_0{snapnum}.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)

logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
sub_prog_url = "http://www.illustris-project.org/api/Illustris-3/snapshots/85/subhalos/185/"
# Get the total number of chunks. Chunk index starts from 0 
# num_chunks = sim['num_files_snapshot']
num_chunks = 600
# initialize halo numbers and subhalo numbers
num_halo = 0
num_subhalo = 0
for chunk_index in range(num_chunks):  # Looping over all chunks
    print(f'sort chunk {chunk_index}')
    
    # Sort particles and return updated numbers
    add_num_halo, add_num_subhalo = sort_halos(snapnum, chunk_number=chunk_index,
                                                FilePath=FilePath, SavePath=SavePath,
                                                Last_Halo_IDs=num_halo, Last_Subhalo_IDs=num_subhalo)
    num_halo += add_num_halo
    num_subhalo += add_num_subhalo

    logging.info(f'Sorted {FilePath}of_subhalo_tab_0{Snap_Index}.{chunk_index}.hdf5')

# Log summary
logging.info('Sorting process completed')

In [ ]:
# Find Total Numbers

# Get the total number of chunks. Chunk index starts from 0 
# num_chunks = sim['num_files_snapshot']
num_chunks = 600
# initialize halo numbers and subhalo numbers
num_halo = 0
num_subhalo = 0
for chunk_index in range(num_chunks):  # Looping over all chunks
    print(f'sort chunk {chunk_index}')
    
    # Sort particles and return updated numbers
    add_num_halo, add_num_subhalo = find_total_numbers(snapnum, chunk_index, 
        FilePath)
    num_halo += add_num_halo
    num_subhalo += add_num_subhalo

print('num halo', num_halo)
print('num subhalo', num_subhalo)

In [ ]:
TargetCat = f'{SavePath}TargetHalo_MergerCat_0{snapnum}.hdf5'
with h5py.File(TargetCat, 'a') as Target_hdf:
    Target_Halo_IDs =  Target_hdf['Group/FOF_Halo_IDs'][:]
    Target_Subhalo_IDs = Target_hdf['Subhalo/Subhalo_IDs'][:]
    Target_Halo_Nsubs = Target_hdf['Group/GroupNsubs'][:]
    Target_HaloM  =  Target_hdf['Group/Group_M_Crit200'][:]
    sub_M = Target_hdf['Subhalo/SubhaloMassInHalfRad'][:]
    Spin = Target_hdf['Subhalo/SubhaloSpin'][:]
    SubhaloVel = Target_hdf['Subhalo/SubhaloVel'][:]
    SubhaloStellarPhotometrics =Target_hdf['Subhalo/SubhaloStellarPhotometrics'][:]
    

In [ ]:
SubhaloStellarPhotometrics[800]